## Fasttext

following tutorial https://fasttext.cc/docs/en/supervised-tutorial.html

In [ ]:
import fasttext

In [ ]:
!wc ./fasttext/cooking.stackexchange.txt


In [ ]:
#splitiing dataset into training and validation sets
!head -n 12404 ./fasttext/cooking.stackexchange.txt > ./fasttext/cooking.train
!tail -n 3000 ./fasttext/cooking.stackexchange.txt > ./fasttext/cooking.valid

In [ ]:
#our first classifier
model = fasttext.train_supervised(input="./fasttext/cooking.train")

In [ ]:
#saving the model
model.save_model("./fasttext/model_cooking.bin")

In [ ]:
model.predict("Which baking dish is best to bake a banana bread ?")

In [ ]:
model.predict("Why not put knives in the dishwasher?")

In [ ]:
model.test("./fasttext/cooking.valid")

In [ ]:
model.test("./fasttext/cooking.valid", k=5)

In [ ]:
model.predict("Why not put knives in the dishwasher?", k=5)

## TODO
- prepare an input file with categories and their keywords
- split dataset in training and validation
- run simple model settings (defaukt as above)

- try to use it as a word vectorizer and measure distance?

In [1]:
## load the data
import sys
# Add src folder to the path
sys.path.append('../src/')
from data.preprocess_keywords import make_cleaned_keywords_df
from data.make_datasets import get_data
# Specify wanted time range
start_date = '2018-01-01'
end_date = '2022-01-01'
# Path to data
data_file = '/home/anya_m/data/dw-project-data/CMS_2010_to_June_2022_ENGLISH.json'
# Load and extract data within time range
df_subset = get_data(data_file, start_date, end_date)
# Cleans keywords and saves data as a dataframe
make_cleaned_keywords_df(df_subset, start_date, end_date)

Getting data...
Loading data DONE. Number of articles is 175659
Extracting data DONE. Number of articles from 2018-01-01 to 2022-01-01 is 50438
Cleaning step 1 out of 2 DONE. Number of unique keywords went from 44649 to 41356
Cleaning step 2 out of 2 DONE. Number of unique keywords went from 41356 to 38097
Finished. Data stored in ../data/interim/clean_keywords_2018-01-01_2022-01-01.json


In [2]:
# Load data
import pandas as pd
# start_date = '2019-01-01'
# end_date = '2022-01-01'
filepath = '../data/interim/clean_keywords_' + start_date + '_' + end_date + '.json'
df = pd.read_json(filepath, orient ='split', compression = 'infer')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50438 entries, 0 to 50437
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            50438 non-null  int64         
 1   lastModifiedDate              50438 non-null  object        
 2   Date                          50438 non-null  datetime64[ns]
 3   keywordStrings                50438 non-null  object        
 4   keywordStringsCleanAfterFuzz  50438 non-null  object        
 5   cleanFocusCategory            50068 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 2.7+ MB


In [17]:
df_clean=df.dropna(how='any')
df_clean_copy = df_clean.copy(deep=True)
##TODO
#df_clean['cleanFocusCategory']=='Law and Justice' replace with Law-and-Justice  and other categories

df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Law and Justice'], 'Law-and-Justice')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Nature and Environment'], 'Nature-and-Environment')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Human Rights'], 'Human-Rights')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Cars and Transportation'], 'Cars-and-Transportation')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Press Freedom'], 'Press-Freedom')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Digital World'], 'Digital-World')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Rule of Law'], 'Rule-of-Law')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Learning German'], 'Learning-German')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Freedom of Speech'], 'Freedom-of-Speech')

/home/anya_m/Documents/git_dw/venv/lib64/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/anya_m/Documents/git_dw/venv/lib64/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/anya_m/Documents/git_dw/venv/lib64/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [38]:
df_clean['cleanFocusCategory'].unique()

array(['Politics', 'Religion', 'Culture', 'Society', 'Law-and-Justice',
       'Business', 'Sports', 'Catastrophe', 'Health', 'Human-Rights',
       'Nature-and-Environment', 'Travel', 'History',
       'Cars-and-Transportation', 'Digital-World', 'Science', 'Media',
       'Technology', 'Education', 'Lifestyle', 'Learning-German',
       'Migration', 'Offbeat', 'Innovation'], dtype=object)

In [20]:
label=df_clean['cleanFocusCategory'].apply(lambda x: ('__label__' +x+' '))
kw=df_clean['keywordStrings'].apply(lambda x: (', '.join(x)))

In [21]:
print(label+kw)

0        __label__Politics Iran, Hassan Rouhani, protes...
1        __label__Politics Tanzania, Africa, New Year, ...
2        __label__Religion Pope Francis, nuclear war, e...
3        __label__Politics Iran protests, Hassan Rouhan...
4        __label__Politics Angela Merkel, coalition tal...
                               ...                        
50433    __label__Politics Putin, New Year, Coronavirus...
50434    __label__Politics Germany, Olaf Scholz, New Ye...
50435    __label__Catastrophe Colorado, wildfires, Boul...
50436    __label__Business RCEP, Regional Comprehensive...
50437    __label__Health France, birth control, contrac...
Length: 50068, dtype: object


In [22]:
from tqdm import tqdm
!rm ./fasttext/output.txt
with open('./fasttext/output.txt', 'a') as f:
    for i, el in tqdm(enumerate(label)):
        #print(i)
        #print(kw[i])
        print(el+kw.iloc[i],file=f)
    
    
    #el.apply(lambda x: ' '.join(kw[i]))

50068it [00:00, 82082.39it/s]


In [23]:
import fasttext

In [24]:
#splitiing dataset into training and validation sets

!head -n 40000 ./fasttext/output.txt > ./fasttext/output.train
!tail -n 15000 ./fasttext/output.txt > ./fasttext/output.valid

In [27]:
#splitiing dataset into training and validation sets
!cat ./fasttext/output.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > ./fasttext/output.preprocessed.txt

!head -n 40000 ./fasttext/output.preprocessed.txt > ./fasttext/output.train
!tail -n 15000 ./fasttext/output.preprocessed.txt > ./fasttext/output.valid

In [28]:
#our first classifier
train_file="./fasttext/output.train"
model = fasttext.train_supervised(input=train_file, lr=1.0, epoch=25, bucket=200000, dim=50, loss='hs')

Read 0M words
Number of words:  26047
Number of labels: 24
Progress: 100.0% words/sec/thread: 1847993 lr:  0.000000 avg.loss:  0.851920 ETA:   0h 0m 0s100.0% words/sec/thread: 1848018 lr: -0.000004 avg.loss:  0.851920 ETA:   0h 0m 0s


In [29]:
#saving the model
model.save_model("./fasttext/model_output.bin")

In [32]:
model.predict("house",k=5)

(('__label__politics',
  '__label__law-and-justice',
  '__label__business',
  '__label__culture'),
 array([0.94289225, 0.05249672, 0.0027589 , 0.00186756]))

In [33]:
model.test("./fasttext/output.valid")
#The output are the number of samples (here 14999), 
# the precision at one (0.6968)  69% and 
# the recall at one (0.0541).

(14999, 0.6968464564304286, 0.6968464564304286)

With a few steps, we were able to go from a precision at one of 12.4% to 59.9%. Important steps included:

preprocessing the data ;
changing the number of epochs (using the option -epoch, standard range [5 - 50]) ;
changing the learning rate (using the option -lr, standard range [0.1 - 1.0]) ;
using word n-grams (using the option -wordNgrams, standard range [1 - 5]).

In [40]:
model.predict("knives", k=5)

(('__label__law-and-justice',
  '__label__media',
  '__label__politics',
  '__label__human-rights',
  '__label__society'),
 array([9.00998354e-01, 7.69218579e-02, 1.21143926e-02, 9.97183938e-03,
        2.52286791e-05]))

In [ ]:
# # Skipgram model :
# model = fasttext.train_unsupervised('data.txt', model='skipgram')

# # or, cbow model :
# model = fasttext.train_unsupervised('data.txt', model='cbow')


# print(model.words)   # list of words in dictionary
# print(model['king']) # get the vector of the word 'king'